In [1]:
import findspark
findspark.init()

In [2]:
from pyspark.sql import SparkSession

#Initialize SparkSession
spark = SparkSession.builder.appName("WordCount").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/09/22 03:44:29 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
sc = spark.sparkContext

In [4]:
rdd = sc.parallelize([1,2,3,4,5])

In [5]:
rdd.collect()

[1, 2, 3, 4, 5]

In [6]:
resultRdd = rdd.map(lambda x: x*2)
resultRdd.collect()

[2, 4, 6, 8, 10]

In [7]:
rdd1 = sc.parallelize([1,2,3,4,5])

In [8]:
resultRdd1 = rdd1.flatMap(lambda x:(x,x*2))
resultRdd1.collect()

[1, 2, 2, 4, 3, 6, 4, 8, 5, 10]

In [9]:
rdd2 = sc.parallelize([1,2,3,4,5])


In [10]:
resultRdd2 = rdd2.filter(lambda x: x%2 ==0)
resultRdd2.collect()

[2, 4]

In [11]:
rdd3 = sc.parallelize([(1,2),(3,4),(1,6),(7,8)])

In [12]:
resultRdd3 = rdd3.reduceByKey(lambda x,y:x+y)
resultRdd3.collect()

[(1, 8), (3, 4), (7, 8)]

In [13]:
resultRdd4 = rdd3.groupByKey()
resultRdd4.collect()

[(1, <pyspark.resultiterable.ResultIterable at 0x7f3fe1600290>),
 (3, <pyspark.resultiterable.ResultIterable at 0x7f3fe16002d0>),
 (7, <pyspark.resultiterable.ResultIterable at 0x7f3fe16000d0>)]

In [14]:
for key, values in resultRdd4.collect():
    print(f"Key: {key}, Values: {list(values)}")

Key: 1, Values: [2, 6]
Key: 3, Values: [4]
Key: 7, Values: [8]


In [15]:
letters = ["a","a","b","c"]
rdd4  = sc.parallelize(letters)
resultRdd5 = rdd4.map(lambda x: (x,1))
result = resultRdd5.collect()
rdd5 = sc.parallelize(result)
resultRdd6= rdd5.reduceByKey(lambda x,y: x+y)
resultRdd6.collect()

[('b', 1), ('c', 1), ('a', 2)]

In [16]:
purchasedf=spark.read.csv("/home/labuser/Downloads/Pandas_datasets/purchases.csv")

In [17]:
purchasedf.show()

+------+------+-------+
|   _c0|   _c1|    _c2|
+------+------+-------+
|  null|apples|oranges|
|  June|     3|      0|
|Robert|     2|      3|
|  Lily|     0|      7|
| David|     1|      2|
+------+------+-------+



In [18]:
purchasedf_01=spark.read.option("inferSchema", True).option("header", True).csv("/home/labuser/Downloads/Pandas_datasets/purchases.csv")

In [19]:
purchasedf_01.show()

+------+------+-------+
|   _c0|apples|oranges|
+------+------+-------+
|  June|     3|      0|
|Robert|     2|      3|
|  Lily|     0|      7|
| David|     1|      2|
+------+------+-------+



23/09/22 03:44:57 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , apples, oranges
 Schema: _c0, apples, oranges
Expected: _c0 but found: 
CSV file: file:///home/labuser/Downloads/Pandas_datasets/purchases.csv


In [20]:
purchaserdd = sc.textFile("/home/labuser/Downloads/Pandas_datasets/purchases.csv")

In [21]:
purchaserdd.collect()

[',apples,oranges', 'June,3,0', 'Robert,2,3', 'Lily,0,7', 'David,1,2']

In [41]:
moviedf=spark.read.schema(udfSchema).option("header", True).csv("/home/labuser/Downloads/Pandas_datasets/IMDB-Movie-Data.csv")

In [42]:
moviedf.printSchema()

root
 |-- Rank: integer (nullable = true)
 |-- Title: string (nullable = true)
 |-- Genre: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Director: string (nullable = true)
 |-- Actors: string (nullable = true)
 |-- Year: string (nullable = true)
 |-- Runtime (Minutes): string (nullable = true)
 |-- Rating: string (nullable = true)
 |-- Votes: string (nullable = true)
 |-- Revenue (Millions): double (nullable = true)
 |-- Metascore: double (nullable = true)



In [40]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DoubleType
udfSchema = StructType([StructField("Rank", IntegerType(), True),
                        StructField("Title", StringType(), True),
                        StructField("Genre", StringType(), True),
                        StructField("Description", StringType(), True),
                        StructField("Director", StringType(), True),
                        StructField("Actors", StringType(), True),
                        StructField("Year", StringType(), True),
                        StructField("Runtime (Minutes)", StringType(), True),
                        StructField("Rating", StringType(), True),
                        StructField("Votes", StringType(), True),
                        StructField("Revenue (Millions)", DoubleType(), True),
                        StructField("Metascore", DoubleType(), True)])

In [43]:
moviedf.show()

+----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----------------+------+------+------------------+---------+
|Rank|               Title|               Genre|         Description|            Director|              Actors|                Year|Runtime (Minutes)|Rating| Votes|Revenue (Millions)|Metascore|
+----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----------------+------+------+------------------+---------+
|   1|Guardians of the ...|Action,Adventure,...|A group of interg...|          James Gunn|Chris Pratt, Vin ...|                2014|              121|   8.1|757074|            333.13|     76.0|
|   2|          Prometheus|Adventure,Mystery...|Following clues t...|        Ridley Scott|Noomi Rapace, Log...|                2012|              124|     7|485820|            126.46|     65.0|
|   3|               Split|   

In [52]:
from pyspark.sql.functions import *
moviedf = moviedf.withColumn("Rev_New",col("Revenue (Millions)")*100).withColumn("Batch", lit("Batch3"))

In [45]:
moviedf.show()

+----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----------------+------+------+------------------+---------+
|Rank|               Title|               Genre|         Description|            Director|              Actors|                Year|Runtime (Minutes)|Rating| Votes|Revenue (Millions)|Metascore|
+----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----------------+------+------+------------------+---------+
|   1|Guardians of the ...|Action,Adventure,...|A group of interg...|          James Gunn|Chris Pratt, Vin ...|                2014|              121|   8.1|757074|            333.13|     76.0|
|   2|          Prometheus|Adventure,Mystery...|Following clues t...|        Ridley Scott|Noomi Rapace, Log...|                2012|              124|     7|485820|            126.46|     65.0|
|   3|               Split|   

In [46]:
newdf.rdd.getNumPartitions()

23/09/22 04:28:21 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: Rank, Title, Genre, Description, Director, Actors, Year, Runtime (Minutes), Rating, Votes, Revenue (Millions), Metascore
 Schema: Rank, Title, Genre, Description, Director, Actors, Year, Runtime, Rating, Votes, Revenue, Metascore
Expected: Runtime but found: Runtime (Minutes)
CSV file: file:///home/labuser/Downloads/Pandas_datasets/IMDB-Movie-Data.csv


9

In [47]:
newdf = moviedf.repartition(9)

In [49]:
newdf.write.csv('/home/labuser/Downloads/TestPartition/partitions_01')

In [ ]:
spark_ui_url = f'{spark.jsc.sc().uiWebUrl().get()}/'

In [51]:
sc.uiWebUrl

'http://ip-172-31-8-237.ap-south-1.compute.internal:4040'

In [53]:
import webbrowser
webbrowser.open_new_tab(sc.uiWebUrl)

True

In [58]:
moviedf.createOrReplaceTempView("movie")

In [59]:
result = spark.sql("select * from movie")

In [60]:
type(result)

pyspark.sql.dataframe.DataFrame

In [61]:
result.show()

+----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----------------+------+------+------------------+---------+-----------------+------+
|Rank|               Title|               Genre|         Description|            Director|              Actors|                Year|Runtime (Minutes)|Rating| Votes|Revenue (Millions)|Metascore|          Rev_New| Batch|
+----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----------------+------+------+------------------+---------+-----------------+------+
|   1|Guardians of the ...|Action,Adventure,...|A group of interg...|          James Gunn|Chris Pratt, Vin ...|                2014|              121|   8.1|757074|            333.13|     76.0|          33313.0|Batch3|
|   2|          Prometheus|Adventure,Mystery...|Following clues t...|        Ridley Scott|Noomi Rapace, Log...|             

In [62]:
testdf=moviedf.sort(col("Title").desc())
testdf.show()

+----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----------------+------+------+------------------+---------+------------------+------+
|Rank|               Title|               Genre|         Description|            Director|              Actors|                Year|Runtime (Minutes)|Rating| Votes|Revenue (Millions)|Metascore|           Rev_New| Batch|
+----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----------------+------+------+------------------+---------+------------------+------+
|  75|            Zootopia|Animation,Adventu...|In a city of anth...|        Byron Howard|Ginnifer Goodwin,...|                2016|              108|   8.1|296853|            341.26|     78.0|           34126.0|Batch3|
| 432|         Zoolander 2|              Comedy|Derek and Hansel ...|         Ben Stiller|Ben Stiller, Owen...|         

In [68]:
df = moviedf.select('Title','Rank')
df.show()

+--------------------+----+
|               Title|Rank|
+--------------------+----+
|Guardians of the ...|   1|
|          Prometheus|   2|
|               Split|   3|
|                Sing|   4|
|       Suicide Squad|   5|
|      The Great Wall|   6|
|          La La Land|   7|
|            Mindhorn|   8|
|  The Lost City of Z|   9|
|          Passengers|  10|
|Fantastic Beasts ...|  11|
|      Hidden Figures|  12|
|           Rogue One|  13|
|               Moana|  14|
|            Colossal|  15|
|The Secret Life o...|  16|
|       Hacksaw Ridge|  17|
|        Jason Bourne|  18|
|                Lion|  19|
|             Arrival|  20|
+--------------------+----+
only showing top 20 rows



In [69]:
from datetime import datetime
df=df.withColumn("last_updated_ts",lit(datetime.now()))
df.show()

+--------------------+----+--------------------+
|               Title|Rank|     last_updated_ts|
+--------------------+----+--------------------+
|Guardians of the ...|   1|2023-09-22 05:37:...|
|          Prometheus|   2|2023-09-22 05:37:...|
|               Split|   3|2023-09-22 05:37:...|
|                Sing|   4|2023-09-22 05:37:...|
|       Suicide Squad|   5|2023-09-22 05:37:...|
|      The Great Wall|   6|2023-09-22 05:37:...|
|          La La Land|   7|2023-09-22 05:37:...|
|            Mindhorn|   8|2023-09-22 05:37:...|
|  The Lost City of Z|   9|2023-09-22 05:37:...|
|          Passengers|  10|2023-09-22 05:37:...|
|Fantastic Beasts ...|  11|2023-09-22 05:37:...|
|      Hidden Figures|  12|2023-09-22 05:37:...|
|           Rogue One|  13|2023-09-22 05:37:...|
|               Moana|  14|2023-09-22 05:37:...|
|            Colossal|  15|2023-09-22 05:37:...|
|The Secret Life o...|  16|2023-09-22 05:37:...|
|       Hacksaw Ridge|  17|2023-09-22 05:37:...|
|        Jason Bourn

In [71]:
df_test=df.selectExpr('CAST(last_updated_ts AS DATE) AS test','Title')
df_test.show()
                

+----------+--------------------+
|      test|               Title|
+----------+--------------------+
|2023-09-22|Guardians of the ...|
|2023-09-22|          Prometheus|
|2023-09-22|               Split|
|2023-09-22|                Sing|
|2023-09-22|       Suicide Squad|
|2023-09-22|      The Great Wall|
|2023-09-22|          La La Land|
|2023-09-22|            Mindhorn|
|2023-09-22|  The Lost City of Z|
|2023-09-22|          Passengers|
|2023-09-22|Fantastic Beasts ...|
|2023-09-22|      Hidden Figures|
|2023-09-22|           Rogue One|
|2023-09-22|               Moana|
|2023-09-22|            Colossal|
|2023-09-22|The Secret Life o...|
|2023-09-22|       Hacksaw Ridge|
|2023-09-22|        Jason Bourne|
|2023-09-22|                Lion|
|2023-09-22|             Arrival|
+----------+--------------------+
only showing top 20 rows



In [72]:
df2=df_test.withColumn("MovieName",lit(col("Title")))
df2.show()

+----------+--------------------+--------------------+
|      test|               Title|           MovieName|
+----------+--------------------+--------------------+
|2023-09-22|Guardians of the ...|Guardians of the ...|
|2023-09-22|          Prometheus|          Prometheus|
|2023-09-22|               Split|               Split|
|2023-09-22|                Sing|                Sing|
|2023-09-22|       Suicide Squad|       Suicide Squad|
|2023-09-22|      The Great Wall|      The Great Wall|
|2023-09-22|          La La Land|          La La Land|
|2023-09-22|            Mindhorn|            Mindhorn|
|2023-09-22|  The Lost City of Z|  The Lost City of Z|
|2023-09-22|          Passengers|          Passengers|
|2023-09-22|Fantastic Beasts ...|Fantastic Beasts ...|
|2023-09-22|      Hidden Figures|      Hidden Figures|
|2023-09-22|           Rogue One|           Rogue One|
|2023-09-22|               Moana|               Moana|
|2023-09-22|            Colossal|            Colossal|
|2023-09-2

In [75]:
df3=moviedf.dropDuplicates(["Director","Title"])
df3.count()

1000

In [76]:
moviedf.count()

1000

In [80]:
test_df=moviedf.withColumn("Rating_Value", when((col("Rating")>=0) & (col("Rating")<5), "Average").
                           when((col("Rating")>=5) & (col("Rating")<8), "Good").
                           when((col("Rating")>=8) & (col("Rating")<10), "Best").otherwise("Normal"))
test_df.show()

+----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----------------+------+------+------------------+---------+-----------------+------+------------+
|Rank|               Title|               Genre|         Description|            Director|              Actors|                Year|Runtime (Minutes)|Rating| Votes|Revenue (Millions)|Metascore|          Rev_New| Batch|Rating_Value|
+----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----------------+------+------+------------------+---------+-----------------+------+------------+
|   1|Guardians of the ...|Action,Adventure,...|A group of interg...|          James Gunn|Chris Pratt, Vin ...|                2014|              121|   8.1|757074|            333.13|     76.0|          33313.0|Batch3|        Best|
|   2|          Prometheus|Adventure,Mystery...|Following clues t...|   

In [100]:

from pyspark.sql.functions import *
def concatColumn(column):
    return column+"_Shell"
my_udf = udf(concatColumn, StringType())

test2df = moviedf.withColumn("Concatenated", my_udf(col("Title")))
test2df.show()

+----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----------------+------+------+------------------+---------+-----------------+------+--------------------+
|Rank|               Title|               Genre|         Description|            Director|              Actors|                Year|Runtime (Minutes)|Rating| Votes|Revenue (Millions)|Metascore|          Rev_New| Batch|        Concatenated|
+----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----------------+------+------+------------------+---------+-----------------+------+--------------------+
|   1|Guardians of the ...|Action,Adventure,...|A group of interg...|          James Gunn|Chris Pratt, Vin ...|                2014|              121|   8.1|757074|            333.13|     76.0|          33313.0|Batch3|Guardians of the ...|
|   2|          Prometheus|Adventure,Mys

In [101]:
cadf = moviedf.cache()

In [102]:
cadf.show(2)

+----+--------------------+--------------------+--------------------+------------+--------------------+----+-----------------+------+------+------------------+---------+-------+------+
|Rank|               Title|               Genre|         Description|    Director|              Actors|Year|Runtime (Minutes)|Rating| Votes|Revenue (Millions)|Metascore|Rev_New| Batch|
+----+--------------------+--------------------+--------------------+------------+--------------------+----+-----------------+------+------+------------------+---------+-------+------+
|   1|Guardians of the ...|Action,Adventure,...|A group of interg...|  James Gunn|Chris Pratt, Vin ...|2014|              121|   8.1|757074|            333.13|     76.0|33313.0|Batch3|
|   2|          Prometheus|Adventure,Mystery...|Following clues t...|Ridley Scott|Noomi Rapace, Log...|2012|              124|     7|485820|            126.46|     65.0|12646.0|Batch3|
+----+--------------------+--------------------+--------------------+------